In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, QuantileTransformer
from sklearn.ensemble import BaggingRegressor, StackingRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeCV
from sklearn.feature_selection import RFE
import lzma
import cloudpickle
import os

In [2]:
df = pd.read_pickle('/tmp/all.pkl')

# display(df.loc[df['city2'].isnull()])
# print(df.isnull().sum())
print(df['type'].value_counts())
print(df['structure'].value_counts())

display(df.loc[df['structure'].isnull()].describe())
display(df.loc[~df['structure'].isnull()].describe())
print(df['structure'].isnull().mean())
print(df['structure'].isnull().sum())

宅地(土地と建物)    1773082
宅地(土地)       1695314
中古マンション等      784713
農地            418774
林地            208322
Name: type, dtype: int64
木造                 1305873
ＲＣ                  607919
ＳＲＣ                 233460
鉄骨造                 151263
軽量鉄骨造                90141
ＲＣ、木造                 5734
ブロック造                 4096
鉄骨造、木造                3305
ＳＲＣ、ＲＣ                 912
ＲＣ、鉄骨造                 805
木造、軽量鉄骨造               754
ＲＣ、ブロック造               440
木造、ブロック造               394
ＳＲＣ、鉄骨造                217
鉄骨造、ブロック造              192
鉄骨造、軽量鉄骨造              132
ＲＣ、軽量鉄骨造                78
ＲＣ、鉄骨造、木造               41
ＳＲＣ、木造                  38
鉄骨造、木造、ブロック造            37
鉄骨造、木造、軽量鉄骨造            33
ＲＣ、木造、ブロック造             32
ブロック造、軽量鉄骨造             23
ＲＣ、木造、軽量鉄骨造             17
木造、ブロック造、軽量鉄骨造          10
ＳＲＣ、ブロック造                7
ＲＣ、鉄骨造、軽量鉄骨造             7
ＲＣ、鉄骨造、ブロック造             5
鉄骨造、ブロック造、軽量鉄骨造          4
ＲＣ、ブロック造、軽量鉄骨造           4
ＳＲＣ、ＲＣ、鉄骨造               3
ＲＣ、鉄骨造、木造、ブロック造          1
ＳＲＣ、木造

,city_code,nearest_sta_dist,price,tubo_price,area,m2_price,frontage,floor_area,building_year,front_road_width,building_cov,floor_cov,time
count,2.474225e+06,1.787751e+06,2.474225e+06,1.695314e+06,2.474225e+06,1.695314e+06,1.631147e+06,47001.000000,51855.000000,1.721455e+06,1.682230e+06,1.682230e+06,2.474225e+06
mean,2.138614e+04,3.147618e+01,2.050007e+07,2.830348e+05,7.383158e+02,8.564063e+04,1.647594e+01,164.450756,1996.765847,6.946809e+00,6.008872e+01,1.910167e+02,2.014989e+05
std,1.331480e+04,2.935010e+01,1.227004e+08,5.677828e+05,1.106982e+03,1.718930e+05,1.080000e+01,231.000426,18.855366,4.805502e+00,8.961918e+00,7.769822e+01,4.473016e+02
min,1.101000e+03,0.000000e+00,1.000000e+02,2.000000e+00,1.000000e+01,1.000000e+00,1.000000e-01,5.000000,1945.000000,1.000000e+00,3.000000e+01,5.000000e+01,2.005750e+05
25%,1.120800e+04,1.100000e+01,1.700000e+06,5.200000e+04,1.650000e+02,1.600000e+04,9.800000e+00,95.000000,1982.000000,4.100000e+00,6.000000e+01,2.000000e+02,2.011250e+05
50%,2.120100e+04,2.000000e+01,7.300000e+06,1.300000e+05,2.800000e+02,3.900000e+04,1.350000e+01,110.000000,1997.000000,6.000000e+00,6.000000e+01,2.000000e+02,2.015000e+05
75%,3.220200e+04,4.500000e+01,1.800000e+07,3.000000e+05,7.500000e+02,9.100000e+04,1.950000e+01,135.000000,2015.000000,7.600000e+00,6.000000e+01,2.000000e+02,2.018750e+05
max,4.738200e+04,1.200000e+02,4.800000e+10,6.600000e+07,5.000000e+03,2.000000e+07,5.000000e+01,2000.000000,2023.000000,9.900000e+01,8.000000e+01,1.300000e+03,2.022750e+05


,city_code,nearest_sta_dist,price,tubo_price,area,m2_price,frontage,floor_area,building_year,front_road_width,building_cov,floor_cov,time
count,2.405980e+06,2.359994e+06,2.405980e+06,0.0,2.405980e+06,0.0,1.523941e+06,1.618403e+06,2.315514e+06,1.622023e+06,2.308292e+06,2.308292e+06,2.405980e+06
mean,1.960047e+04,2.207230e+01,3.535199e+07,NaN,1.971787e+02,NaN,1.266616e+01,1.733695e+02,1.996314e+03,6.628892e+00,6.180047e+01,2.231854e+02,2.015079e+05
std,1.087761e+04,2.364924e+01,1.936874e+08,NaN,2.796084e+02,NaN,8.213368e+00,2.518950e+02,1.580960e+01,4.579941e+00,1.067083e+01,1.236057e+02,4.556193e+02
min,1.101000e+03,0.000000e+00,1.000000e+02,NaN,1.000000e+01,NaN,4.000000e-01,5.000000e+00,1.945000e+03,1.000000e+00,3.000000e+01,5.000000e+01,2.005750e+05
25%,1.223600e+04,8.000000e+00,1.100000e+07,NaN,7.000000e+01,NaN,7.600000e+00,9.500000e+01,1.985000e+03,4.000000e+00,6.000000e+01,2.000000e+02,2.011250e+05
50%,1.420500e+04,1.400000e+01,2.200000e+07,NaN,1.150000e+02,NaN,1.100000e+01,1.050000e+02,1.997000e+03,6.000000e+00,6.000000e+01,2.000000e+02,2.015250e+05
75%,2.720500e+04,2.500000e+01,3.500000e+07,NaN,2.000000e+02,NaN,1.500000e+01,1.350000e+02,2.010000e+03,6.700000e+00,6.000000e+01,2.000000e+02,2.019000e+05
max,4.738200e+04,1.200000e+02,1.700000e+11,NaN,2.000000e+03,NaN,5.000000e+01,2.000000e+03,2.023000e+03,9.990000e+01,8.000000e+01,1.300000e+03,2.022750e+05


0.506992021851541
2474225


In [3]:
# available features
features = [
    'type',
    # 'zone_type',
    'city_plan',
    'city_code',
    'prefecture',
    'city',
    'city2',
    'nearest_sta',
    'nearest_sta_dist',
    'age',
    # 'structure',
    'area',
    'floor_area',
    # 'layout', # many nan
    # 'front_road_dir',
    # 'front_road_type',
    'front_road_width',
    # 'building_cov',
    # 'floor_cov',
    'building_year',
    'time',
    # 'choume'
    # 'sawa',
    # 'numa',
    # 'tani',
    # 'ken',
    'city_code_low',
    # 'city_code_low2',
    # 'city2_last',
    # 'city2_first',
    # 'city_last',
    # 'city_first',
    # 'city2_len',
    # 'city_len',
    # 'floor_area_area',
    # 'area_front_road_width',
    # 'area_front_road_width2',
]

In [4]:
def show_importance(model, features):
    importances = model.feature_importances_
    feature_imp = pd.DataFrame(zip(importances, features), columns=['value', 'feature'])
    feature_imp = feature_imp.sort_values('value')
    display(feature_imp)

In [5]:
class RemlModel:
    def __init__(self, df_all):
        self._city_code_map = df_all.groupby(['prefecture', 'city'])[['city_code']].nth(0)
        self._dtypes = df_all.dtypes

    def fit(self, df, mean_only=False):
        df = df.copy()
        
        cat_features = [
            'type',
            'city_plan',
            'prefecture',
            'city',
            'city2',
            'nearest_sta',
            # 'structure',
            # 'layout',
            # 'front_road_dir',
            # 'front_road_type',
            # 'city2_last',
            # 'city2_first',
            # 'city_last',
            # 'city_first',
        ]
        num_features = [x for x in features if x not in cat_features]
        # num_features = [features.index(x) for x in num_features]
        # cat_features = [features.index(x) for x in cat_features]
        
        def create_model(alpha=None):
            n_est_scale = 10
            
            model = lgb.LGBMRegressor(
                alpha=alpha,
                objective='regression' if alpha is None else 'quantile',
                n_estimators=100 * n_est_scale,
                learning_rate=0.1 / n_est_scale,
                # colsample_bytree=0.5,
                subsample=0.5,
                subsample_freq=1,
                # extra_trees=True,
                importance_type='gain',
                random_state=1, 
                n_jobs=-1
            )
            et = lgb.LGBMRegressor(
                n_estimators=100 * n_est_scale,
                learning_rate=0.1 / n_est_scale,
                colsample_bytree=0.5,
                subsample=0.5,
                subsample_freq=1,
                extra_trees=True,
                random_state=1, 
                n_jobs=-1
            )
            rf = lgb.LGBMRegressor(
                n_estimators=100 * n_est_scale,
                learning_rate=0.1 / n_est_scale,
                boosting_type='rf',
                colsample_bytree=0.5,
                subsample=0.5,
                subsample_freq=1,
                # extra_trees=True,
                random_state=1, 
                n_jobs=-1
            )
            
            xg = xgb.XGBRegressor(
                random_state=1,
                n_jobs=-1,
            )
            cb = CatBoostRegressor(
                verbose=0,
                random_state=1,
                thread_count=-1,
            )
            
            num_transformer = Pipeline([
                ("imputer", SimpleImputer(strategy="median")), 
                # ("scaler", StandardScaler()),
                ("qt", QuantileTransformer(output_distribution='normal', random_state=1)),
                ("fe", FeatureUnion([
                    ('pt', 'passthrough'),
                    ('pca', PCA(n_components=2)),
                ])),
            ])

            cat_transformer = Pipeline([
                ("le", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, encoded_missing_value=np.nan)),
            ])
            
            ridge = Pipeline([
                ('ct', ColumnTransformer([
                    ("num", num_transformer, np.arange(len(num_features))),
                ])),
                ('model', RidgeCV()),
            ])
            
            # model = RFE(model)
            # model = BaggingRegressor(model, n_estimators=10, random_state=1)
            # model = StackingRegressor([('m', model)], final_estimator=model, cv=None, n_jobs=None, passthrough=True, verbose=0) # good
            model = StackingRegressor([
                ('m', model),
                ('qt_m', TransformedTargetRegressor(model, transformer=QuantileTransformer(output_distribution='normal', random_state=1))),
                ('et', et),
                ('rf', rf),
                ('xg', xg), # good
                ('cb', cb), # good
                ('ridge', ridge),
            ], final_estimator=model, cv=None, n_jobs=None, passthrough=True, verbose=0) # good

            model = Pipeline([
                ('ct', ColumnTransformer([
                    ("num", 'passthrough', num_features),
                    # ("num", num_transformer, num_features),
                    ("cat", cat_transformer, cat_features),
                ])),
                ('model', model),
            ])

            # model = BaggingRegressor(model, n_estimators=10, random_state=1)
            
            return model
        
        df = df.sample(frac=1, random_state=1) # for ordinal encoder
        
        # y = np.log(df['price'])
        y = np.log(df['price'] / df['area'])
        # plt.hist(y, bins=100)
        # plt.show()
        X = self.calc_features(df)
        
        model = create_model()
        model.fit(X, y)
        
        # show_importance(model.named_steps['model'], num_features + cat_features)
        # show_importance(model.named_steps['model'].estimators_[0], num_features + cat_features)
        
        self._model = model
        
        self.mean_only = mean_only
        if not mean_only:
            model = create_model(alpha=0.1)
            model.fit(X, y)
            self._model_10 = model
            model = create_model(alpha=0.9)
            model.fit(X, y)
            self._model_90 = model
    
    def predict(self, df):
        df = df.copy()
        
        for col in df.columns:
            if col in self._dtypes:
                df[col] = df[col].astype(self._dtypes[col])
        
        X = self.calc_features(df)
        # return np.exp(self._model.predict(X))
        
        df_res = pd.DataFrame(index=df.index)
        df_res['price'] = np.exp(self._model.predict(X)) * df['area'].values
        if not self.mean_only:
            df_res['price_10'] = np.exp(self._model_10.predict(X)) * df['area'].values
            df_res['price_90'] = np.exp(self._model_90.predict(X)) * df['area'].values
        # df_res['price'] = np.exp(self._model.predict(X))
        # df_res['price_10'] = np.exp(self._model_10.predict(X))
        # df_res['price_90'] = np.exp(self._model_90.predict(X))
        
        return df_res
    
    def metadata(self):
        return {
            'unique_values': self._unique_values,
        }
    
    def calc_features(self, df):
        df_orig = df.reset_index()
        df = df_orig.copy()
        
        df = df.merge(self._city_code_map, how='inner', on=['prefecture', 'city'])
        df = df.set_index('index').loc[df_orig['index']]
        
#         from kanjize import kanji2number
#         def conv_kanji(x):
#             if isinstance(x, float):
#                 return 0
#             print(x)
#             return kanji2number(x)
        
#         df['choume'] = df['city2'].str.extract(r'([一二三四五六七八九十]+)丁目')
#         df['choume'] = df['choume'].apply(conv_kanji).astype('int')
        
        # df['sawa'] = df['city2'].str.contains('沢')
        # df['numa'] = df['city2'].str.contains('沼')
        # df['tani'] = df['city2'].str.contains('谷')

        # df['ken'] = df['prefecture'].str.contains('県')
        df['city_code_low'] = df['city_code'] % 1000
        # df['city_code_low2'] = df['city_code'] % 100
    
        df['age'] = df['time'] - df['building_year'] * 100
        
        df['city2_last'] = df['city2'].str[-1:]
        df['city2_first'] = df['city2'].str[:1]
        df['city_last'] = df['city'].str[-1:]
        df['city_first'] = df['city'].str[:1]
        df['city2_len'] = df['city2'].str.len()
        df['city_len'] = df['city'].str.len()
        
        df['floor_area_area'] = df['floor_area'] / df['area']
        df['area_front_road_width'] = df['area'] / df['front_road_width']
        df['area_front_road_width2'] = df['area'] / df['front_road_width'] ** 2

        # display(df)
        # display(df_orig)
        
        df = df[features]

        return df

In [6]:
df = pd.read_pickle('/tmp/all.pkl')
# df = df.loc[df['type'] == '宅地(土地)']
# df = df.loc[df['type'] == '宅地(土地と建物)']
# df = df.loc[df['type'] == '中古マンション等']
# df = df.loc[df['type'] == '農地']
# df = df.loc[df['type'] == '林地']
# df = df.loc[df['type'].isin(['宅地(土地と建物)', '中古マンション等'])]
df = df.loc[df['type'].isin(['宅地(土地)', '宅地(土地と建物)', '中古マンション等'])]

model = RemlModel(df)

# df = df.iloc[::100]
df = df.drop(columns='city_code')

model.fit(df)
# print(model.metadata())

data = cloudpickle.dumps(model)
data = lzma.compress(data)
model_name = '20230221_eda'
with open('/home/jovyan/data/{}.xz'.format(model_name), 'wb') as f:
    f.write(data)
print('model size {}'.format(os.path.getsize('/home/jovyan/data/{}.xz'.format(model_name))))

cv = KFold(shuffle=True, random_state=1)

def calc_score(y, y_pred):
    return r2_score(np.log(y), np.log(y_pred))
    # return pearsonr(np.log(y), np.log(y_pred))[0]

scores = []
for train_idx, val_idx in cv.split(df):
    model.fit(df.iloc[train_idx], mean_only=True)
    
    df_val = df.iloc[val_idx]
    df_res = model.predict(df_val)
    y_pred = df_res['price']
    y = df_val['price']
    score = calc_score(y, y_pred)
    scores += [{ 'score': score, 'type': 'all' }]
    print('score {}'.format(score))
    
    for tp, df_type in df_val.groupby('type'):
        score = calc_score(y.loc[df_type.index], y_pred.loc[df_type.index])
        scores += [{ 'score': score, 'type': tp }]
        print('{} score {}'.format(tp, score))

df_score = pd.DataFrame(scores)
display(df_score.groupby('type')[['score']].describe())

KeyboardInterrupt: 

In [278]:
def create_metadata(df):
    unique_values = {}
    for col in df.columns:
        unique_values[col] = df[col].sort_values().unique().tolist()

    x = df['prefecture']
    x = x + ':' + df['city']
    x = x + ':' + df['city2']
    x = x + ':' + df['nearest_sta']
    unique_values['prefecture_city_city2_nearest_sta'] = x.sort_values().unique().tolist()
    
    return {
        'unique_values': unique_values,
    }

df = pd.read_pickle('/tmp/all.pkl')
df = df.loc[df['type'].isin(['宅地(土地)', '宅地(土地と建物)', '中古マンション等'])]
metadata = create_metadata(df)

data = cloudpickle.dumps(metadata)
data = lzma.compress(data)
with open('/home/jovyan/data/{}_metadata.xz'.format(model_name), 'wb') as f:
    f.write(data)